In [1]:
import matplotlib.pyplot as plt
import numpy as np

import camb

from scipy.optimize import curve_fit

In [3]:
# Load Data
pars     = np.load('CL_data/parameter_cross1.npy')  # [H0, ombh2, omch2 ] x 100
lensed   = np.load('CL_data/lensed_cross1.npy')     # [C_2, ..., C_5000] x 100 (lensed)
unlensed = np.load('CL_data/unlensed_cross1.npy')     # [C_2, ..., C_5000] x 100 (unlensed)

In [8]:
omcs = pars[:,1]
As = pars[:,2]
ombs = np.ones(np.shape(omcs)) * 0.0224

ells = np.array([l for l in range(2, 5000)])

In [15]:
#getting dependent variables
lensing = lensed/unlensed #lensing tail
camb_data_format = []
for i in range(len(lensing)):
    for j in range(len(lensing[i])):
        camb_data_format.append(lensing[i][j])

In [9]:
fid_ombh2 = 0.0224
fid_omch2 = 0.12

In [34]:
def func(X, b1):
    #expected form of the lensing tail function for curve fitting
    ell = X[0]
    ommh2 = (X[1]/0.1424) - 1
    ombh2 = X[2]/ 0.0224 - 1
    As = X[3] / 3.043 - 1

    beta_1 = 0.6838 * (1 + -0.14316  * ommh2  + 0.21621 * ommh2**2 + -0.04256699 * ombh2 + 0.00295 * As + 0.0772 * As**2 + b1 * ommh2 * As) 
    beta_2 = 2803.1562 * (1 + -0.1925  * ommh2 + 0.33984  * ommh2**2 + 0.01274244 * ombh2 + 0.00465607 * ombh2**2 +0.01288  * As + -0.1299 * As**2) 
    
    beta_3 = 3752.098 * (1 + -0.49117 * ommh2 + 0.3637 * ommh2**2 + 0.4959597  * ombh2 + 0.25204221 * ombh2**2 + -0.1017 * As + 0.1915  * As**2)
    beta_4 = 407.009 * (1 + -0.879 * ommh2 + -0.3132 *ommh2**2 + 0.39838899 * ombh2 + -0.69752433 * ombh2**2 + 0.5307 * As + 0.1483 * As**2 ) 
    alpha = 2.5867 *(1 + 0.7702 * ommh2 + 0.3882 * ommh2 **2  + -0.98242872 * ombh2 + 1.15253038 * ombh2**2 + 1.4088  * As + 0.7482    * As**2 )
        
    sigm = (1 + np.exp(-(ell-beta_3)/beta_4))**(-1)

    poly = (beta_1*(ell/beta_2)**alpha - 1)
    
    L = poly*sigm + 1
    
    return L

In [35]:
cosmos = [ombs[i] + omcs[i] for i in range(len(ombs))]    #x3 variable
n = len(cosmos) #number of cosmologies
params = np.zeros((len(ells) * n, 4))   #ind variables
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),0] = ells
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),1] = cosmos[i]  #ommh2
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),2] = ombs[i]  #ombh2
for i in range(n):
    params[(i) * len(ells): (i+1)*len(ells),3] = As[i]

In [36]:
x1 = params[:,0]
x2 = params[:,1]
x3 = params[:,2]
x4 = params[:,3]

In [37]:
answers, _ = curve_fit(func,(x1, x2, x3, x4),camb_data_format)  #doing the curve fitting

In [38]:
answers

array([-2.75811397])